# Brugia Viral Search

The goal of this analysis is to identify and hopefully assemble to some degree the viruses that are found present in the Brugia RNA-Seq data.

The first step is to run kraken on the data to identify likely viruses from the RefSeq database. Given it is limited to the RefSeq database, it is surely not exhastive enough to be extrememly beneficial in de novo assembly, so LMAT is going to be run as well.